# Assignment : Spam Filter
## Description
This assignment is near-final modulo some small adjustments (6 Nov '16)
In this assignment, you will discover that in many practical machine learning problems implementing the learning algorithm is often only a small part of the overall system. Thus to get a high mark for this assignment you need to implement any of the more advanced classification techniques or clever pre-processing methods. You will find plenty of them on the internet. If you do not know where to look for them, ask Google ;-).

Here your task is to build the standard (i.e. multinomial) Naive Bayes text classifier described during the lectures. You should test your program using the automatic marking software (described below), so it is critically important that it follows the specifications in detail.

You will train your classifier on real-world e-mails, which you can download from here. Each training e-mail is stored in a separate file. The names of spam training e-mails start with spam, while the names of ham e-mails start with ham.

## Marking criteria

Part 1 (40%):
    - Your program classifies the testing set with an accuracy significantly higher than random within 30 minutes
    - Use very simple data preprocessing so that the emails can be read into the Naive Bayes (remove everything else other than words from emails)
    - Write simple Naive Bayes multinomial classifier or use an implementation from a library of your choice
    - Classify the data
    - Report your results with a metric (e.g. accuracy) and method (e.g. cross validation) of your choice
    - Choose a baseline and compare your classifier against it

Part 2 (30%):
    - Use some smart feature processing techniques to improve the classification results
    - Compare the classification results with and without these techniques
    - Analyse how the classification results depend on the parameters (if available) of chosen techniques
    - Compare (statistically) your results against any other algorithm of your choice (use can use any library); compare and contrast results, ensure fair comparison

Part 3 (30%):
    - Calibration (15%): calibrate Naive Bayes probabilities, such that they result in low mean squared error
    - Naive Bayes extension (15%): modify the algorithm in some interesting way (e.g. weighted Naive Bayes)
    

** Convert from .ipynb to .py : ** $ ipython nbconvert --to python filter.ipynb



In [3]:
# Import all necessary packages
import math
import numpy as np
import pandas as pd

import os, re




** Part 1 Filtering

In [7]:
import email

# Removes header from email if present
def get_body():
    if msg.is_multipart():
        for payload in msg.get_payload():
            # if payload.is_multipart(): ...
            return payload.get_payload()
    else:
        return msg.get_payload()

# Returns text containing only words (lowercase)
def remove_extras(text):
    brackets = "\([^)]*\)"
    email_address = "([\w.-]+)@([\w.-]+)"
    web_address = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    numbers = "\d+"
    alphanumeric = "([^\s\w]|_)+"
    whitespace = "\s+"

    text = re.sub(email_address, '', text)
    text = re.sub(web_address, '', text)
    text = re.sub(numbers, '', text)
    text = re.sub(alphanumeric, '', text)
    text = re.sub(whitespace, ' ', text).strip()
    text = text.lower() # Lowercase
    return text


def filter_part1(email): 
    message = get_body()
    message = remove_extras(message)
    return message


# ---------------------



with open('training_data/ham068.txt') as f:
    text_test = f.read()

msg = email.message_from_string(text_test)

print(filter_part1(msg))







i have translated the article in full see end of post i think that ive done a far better job than the google translation at least its readable now any corrections appreciated stuff in s is my and others additions to the debate my apologies if ive paraphrased anybody incorrectly i will be glad to retract if anyone is miffed the article makes four main points absence of critical clauses in this case the idea is that the licence is invalid because it doesnt specify under what countrys law the gpl is governed specification in english only that for the end user as opposed to businesses the gpl doesnt apply because its not written in french arbitary licence change the point here is that under french law the author can change the terms of the licence arbitarily this is because any granting of rights by an author must be clearly delimited in terms of how long where to whom dates and times in the absence of such limitations the original author has the right to change his software back to closed

** Part 1 Filtering

In [105]:
from email.header import decode_header
def getheader(header_text, default="ascii"):
    """Decode the specified header"""

    headers = decode_header(header_text)
    header_sections = [unicode(text, charset or default)
                       for text, charset in headers]
    return u"".join(header_sections)


getheader(msg["from"])

u'Paul Linehan<plinehan@yahoo.com>'